In [0]:
#import modules we need to use for this session
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
# PCA module
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import r2_score

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Task 1. House price prediction using 311 data

### Can 311 complaining profile of urban neighborhoods enhance real-estate price prediction?

[Zhu, E., & Sobolevsky, S. (2018). House Price Modeling with Digital Census. arXiv preprint arXiv:1809.03834.]

Before we tried to use some parameters of the house (size) to predict its price. However notice that location matters as well. Below we upload a sample of more than 14000 individual house sales all around NYC in 2012. Each record together with the parameters of the house also contains important characteristics of the location (zip code) - average income of its residents (accodring to US census), as well as the relative structure of 311 complaints happening in the area.

In [0]:
REsample=pd.read_csv('/content/drive/My Drive/Untitled folder/NYChome_sales2012.csv',index_col=0)

In [0]:
len(REsample)

14611

In [0]:
REsample.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date
204018,4,LAURELTON,12941,94,NaN,A2,131-43 FRANCIS LEWIS BLVD,11413,0,3560,1473,1940,0,2012-01-01
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05
217598,2,BAYCHESTER,4707,70,NaN,A5,1134 EAST 213 STREET,10469,0,2320,1813,1950,335700,2012-05-17
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03
217600,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-12-20


In [0]:
REsample.columns

Index([u'borough', u'neighborhood', u'block', u'lot', u'easement',
       u'building_class_present', u'address', u'zip_code',
       u'commmercial_units', u'land_sq_feet', u'gross_sq_feet', u'year_built',
       u'sale_price', u'sale_date'],
      dtype='object')

In [0]:
#how many houses do we have per each borough?
np.bincount(REsample.borough)

array([   0,  216,  894, 2769, 7288, 3444])

In [0]:
#split into training (1), validation (2) and test (3) samples
np.random.seed(2018)
splitind=np.random.uniform(size=len(REsample)) #draw samples within low and high limit, each sample is likely to be drawn 
splitind

array([0.88234931, 0.10432774, 0.90700933, ..., 0.86845249, 0.57514979,
       0.79229244])

In [0]:
REsample1=REsample.loc[splitind<=0.4] #training
REsample2=REsample.loc[(splitind>0.4)&(splitind<0.7)] #validation
REsample3=REsample.loc[(splitind>=0.7)] #test

In [0]:
len(REsample1) #loosing around 50 records within sparse zip codes

5832

In [0]:
len(REsample2)

4394

In [0]:
len(REsample3)

4385

In [0]:
# now load and curate 311 data which we're going to use in order to characterize the location

In [0]:
data311 = pd.read_csv('/content/drive/My Drive/Untitled folder/aggr311.csv' , index_col=0 )
data311.head()

,Zip,Complain,Count
1,NaN,Adopt-A-Basket,5
2,10001,Adopt-A-Basket,1
3,10003,Adopt-A-Basket,1
4,10009,Adopt-A-Basket,1
5,10010,Adopt-A-Basket,1


In [0]:
data311.Zip=pd.to_numeric(data311.Zip,errors='coerce') #Take separate series and convert to numeric, coercing when told to, errors = 'coerce' means then invalid parsing will be set as NaN
data311.head()

,Zip,Complain,Count
1,NaN,Adopt-A-Basket,5
2,10001.0,Adopt-A-Basket,1
3,10003.0,Adopt-A-Basket,1
4,10009.0,Adopt-A-Basket,1
5,10010.0,Adopt-A-Basket,1


In [0]:
data311=data311.loc[(data311.Zip>=10000)&(data311.Zip<11500)] #take only NYC zip codes

In [0]:
data311=pd.pivot_table(data311,index='Zip',columns='Complain',values='Count',fill_value=0)

In [0]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0,0,1,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,2,0,0,0,0,1,3,0,0,0,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,1,0,0,0,16,1,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0
10001.0,0,1,90,0,0,1,20,36,1,1,2,51,2,1,0,498,1,0,33,3,0,0,3,1,3,394,18,0,5,2,7,7,4,7,3,108,0,7,1,98,...,2,7,71,49,29,9,38,0,38,0,0,0,5,0,429,4,14,1,16,0,1,0,707,39,1,0,0,0,0,0,3,0,2,51,5,5,2,366,0,0
10002.0,15,0,80,0,0,21,18,20,2,6,8,73,8,0,24,921,2,0,120,4,2,0,4,2,0,167,4,4,9,5,30,14,20,17,8,144,4,14,1,0,...,19,2,102,50,27,7,26,0,65,0,0,1,8,0,408,296,32,8,14,1,4,4,368,28,319,0,0,1,0,7,5,1,4,27,14,10,7,324,0,0
10003.0,15,1,143,0,2,44,24,13,2,2,14,69,16,0,0,1461,1,0,67,4,4,2,4,3,3,181,4,3,17,5,45,11,33,17,16,162,5,21,0,25,...,17,5,115,65,21,7,45,0,49,1,0,0,3,1,572,32,10,16,16,0,5,0,600,20,23,0,0,2,1,7,7,11,2,54,25,12,5,318,0,0
10004.0,0,0,15,0,0,4,3,1,0,1,1,8,3,0,0,123,0,0,3,0,0,0,2,0,0,55,0,2,2,0,1,1,0,1,1,12,0,0,0,19,...,5,0,32,23,8,1,8,0,3,0,0,0,1,0,133,93,39,17,21,0,0,0,142,10,96,0,1,0,0,0,1,1,0,16,21,0,0,37,0,0


In [0]:
data311.shape

(264, 162)

In [0]:
list_311=list(data311.columns)
list_311[:10]

['APPLIANCE',
 'Adopt-A-Basket',
 'Air Quality',
 'Animal Abuse',
 'Animal Facility - No Permit',
 'Animal in a Park',
 'Asbestos',
 'BEST/Site Safety',
 'Beach/Pool/Sauna Complaint',
 'Bike Rack Condition']

In [0]:
Total311=data311.sum(axis=1) #total 311 activity per zip code
Total311.head()

Zip
10000.0      114
10001.0     6829
10002.0    11276
10003.0    11269
10004.0     1540
dtype: int64

In [0]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0,0,1,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,2,0,0,0,0,1,3,0,0,0,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,1,0,0,0,16,1,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0
10001.0,0,1,90,0,0,1,20,36,1,1,2,51,2,1,0,498,1,0,33,3,0,0,3,1,3,394,18,0,5,2,7,7,4,7,3,108,0,7,1,98,...,2,7,71,49,29,9,38,0,38,0,0,0,5,0,429,4,14,1,16,0,1,0,707,39,1,0,0,0,0,0,3,0,2,51,5,5,2,366,0,0
10002.0,15,0,80,0,0,21,18,20,2,6,8,73,8,0,24,921,2,0,120,4,2,0,4,2,0,167,4,4,9,5,30,14,20,17,8,144,4,14,1,0,...,19,2,102,50,27,7,26,0,65,0,0,1,8,0,408,296,32,8,14,1,4,4,368,28,319,0,0,1,0,7,5,1,4,27,14,10,7,324,0,0
10003.0,15,1,143,0,2,44,24,13,2,2,14,69,16,0,0,1461,1,0,67,4,4,2,4,3,3,181,4,3,17,5,45,11,33,17,16,162,5,21,0,25,...,17,5,115,65,21,7,45,0,49,1,0,0,3,1,572,32,10,16,16,0,5,0,600,20,23,0,0,2,1,7,7,11,2,54,25,12,5,318,0,0
10004.0,0,0,15,0,0,4,3,1,0,1,1,8,3,0,0,123,0,0,3,0,0,0,2,0,0,55,0,2,2,0,1,1,0,1,1,12,0,0,0,19,...,5,0,32,23,8,1,8,0,3,0,0,0,1,0,133,93,39,17,21,0,0,0,142,10,96,0,1,0,0,0,1,1,0,16,21,0,0,37,0,0


In [0]:
data311=data311.div(data311.sum(axis=1), axis=0) #normalize activity of various categories within zip code by total
data311=data311.loc[Total311>100] #keep only those zip codes having sufficient activity
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...

In [0]:
len(data311) #only 179 zip codes with reliable data left

179

In [0]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...

In [0]:
#data311.columns  = data311.columns.str.lower()
#data311.columns  =data311.columns.str.split(' ').str.get(0)

In [0]:
#data311.columns  =data311.columns.str.split('-').str.get(0)
#data311.columns  =data311.columns.str.split('/').str.get(0)

In [0]:
data311.columns = ['F%d'%p for p in range(0,len(data311.columns))]

In [0]:
data311.head()

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...,0.001509,0.000444,0.010205,0.005768,0.001864,0.000621,0.003993,0.0,0.004348,0.000089,0.0,0.000000,0.000266,0.000089,0.050759,0.002840,0.000887,0.001420,0.001420,0.000000,0.000444,0.000000,0.053243,0.001775,0.002041,0.0,0.000000,0.000177,0.000089,0.000621,0.000621,0.000976,0.000177,0.004792,0.002218,0.001065,0.000444,0.028219,0.0,0.0
10004.0,0.000000,0.000000,0.009740,0.0,0.000000,0.002597,0.001948,0.000649,0.000000,0.000649,0.000649,0.005195,0.001948,0.000000,0.000000,0.079870,0.000000,0.0,0.001948,0.000000,0.000000,0.000000,0.001299,0.000000,0.000000,0.035714,0.000000,0.001299,0.001299,0.000000,0.000649,0.000649,0.000000,0.000649,0.000649,0.007792,0.000000,0.000000,0.000000,0.012338,...,0.003247,0.000000,0.020779,0.014935,0.005195,0.000649,0.005195,0.0,0.001948,0.000000,0.0,0.000000,0.000649,0.000000,0.086364,0.060390,0.025325,0.011039,0.013636,0.000000,0.000000,0.000000,0.092208,0.006494,0.062338,0.0,0.000649,0.000000,0.000000,0.000000,0.000649,0.000649,0.000000,0.010390,0.013636,0.000000,0.000000,0.024026,0.0,0.0


In [0]:
data311.index.rename('zip_code', inplace = True)

In [0]:
data311.head()

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
zip_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...,0.001509,0.000444,0.010205,0.005768,0.001864,0.000621,0.003993,0.0,0.004348,0.000089,0.0,0.000000,0.000266,0.000089,0.050759,0.002840,0.000887,0.001420,0.001420,0.000000,0.000444,0.000000,0.053243,0.001775,0.002041,0.0,0.000000,0.000177,0.000089,0.000621,0.000621,0.000976,0.000177,0.004792,0.002218,0.001065,0.000444,0.028219,0.0,0.0
10004.0,0.000000,0.000000,0.009740,0.0,0.000000,0.002597,0.001948,0.000649,0.000000,0.000649,0.000649,0.005195,0.001948,0.000000,0.000000,0.079870,0.000000,0.0,0.001948,0.000000,0.000000,0.000000,0.001299,0.000000,0.000000,0.035714,0.000000,0.001299,0.001299,0.000000,0.000649,0.000649,0.000000,0.000649,0.000649,0.007792,0.000000,0.000000,0.000000,0.012338,...,0.003247,0.000000,0.020779,0.014935,0.005195,0.000649,0.005195,0.0,0.001948,0.000000,0.0,0.000000,0.000649,0.000000,0.086364,0.060390,0.025325,0.011039,0.013636,0.000000,0.000000,0.000000,0.092208,0.006494,0.062338,0.0,0.000649,0.000000,0.000000,0.000000,0.000649,0.000649,0.000000,0.010390,0.013636,0.000000,0.000000,0.024026,0.0,0.0


#### Q1. Run linear regression 'sale_price ~ gross_sq_feet+land_sq_feet+year_built' over the training data

## REsample1 is the training data

In [0]:
temp_REsample1 = REsample1.copy()

In [0]:
REsample1.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03
217605,2,BAYCHESTER,4715,61,NaN,A9,1118 221ST STREET,10469,0,2514,1690,1950,250000,2012-12-17
217609,2,BAYCHESTER,4724,21,NaN,A1,3453 KINGSLAND AVENUE,10469,0,2283,1588,1910,249600,2012-06-01
217610,2,BAYCHESTER,4725,45,NaN,A9,1453 HICKS STREET,10469,0,1710,1690,1945,313000,2012-09-20


In [0]:
REsample1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 217555 to 301570
Data columns (total 14 columns):
borough                   5832 non-null int64
neighborhood              5832 non-null object
block                     5832 non-null int64
lot                       5832 non-null int64
easement                  0 non-null float64
building_class_present    5832 non-null object
address                   5832 non-null object
zip_code                  5832 non-null int64
commmercial_units         5832 non-null int64
land_sq_feet              5832 non-null int64
gross_sq_feet             5832 non-null int64
year_built                5832 non-null int64
sale_price                5832 non-null int64
sale_date                 5832 non-null object
dtypes: float64(1), int64(9), object(4)
memory usage: 683.4+ KB


In [0]:
lm  = smf.ols(formula = 'sale_price ~ gross_sq_feet+land_sq_feet+year_built', data = temp_REsample1).fit()
temp_REsample1['Prediction'] = lm.predict()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:             sale_price   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     342.8
Date:                Sat, 16 Nov 2019   Prob (F-statistic):          5.23e-205
Time:                        05:44:02   Log-Likelihood:                -88308.
No. Observations:                5832   AIC:                         1.766e+05
Df Residuals:                    5828   BIC:                         1.767e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      7.637e+06   8.92e+05      8.561

In [0]:
print('The R2 Score over Training Data is: {}'.format(r2_score(temp_REsample1.sale_price, temp_REsample1.Prediction)))

The R2 Score over Training Data is: 0.149987575604


#### Q2. Add 311 data corresponding to the zip code of the house location to REsample1,2,3 
in order to characterize the house's neighborhood by merging the dataframes REsample1,2,3 with data311 by zip code. Name merged dataframes REsample1_311, REsample2_311, REsample3_311 
rename 311 columns to F1,F2,... for easier formulae engineering
but keep the original names as a dictionary {'F1':'APPLIANCE', ...}

In [0]:
REsample1_311 = REsample1.merge(data311, on = 'zip_code')

In [0]:
REsample1_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
0,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
1,2,MOUNT HOPE/MOUNT EDEN,2790,53,NaN,A1,1693 CLAY AVENUE,10457,0,3610,1998,1901,168324,2012-06-28,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
2,2,MOUNT HOPE/MOUNT EDEN,2791,36,NaN,A1,1690 MONROE AVENUE,10457,0,2375,1617,1910,250000,2012-12-28,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
3,2,MOUNT HOPE/MOUNT EDEN,2802,33,NaN,A1,258 MOUNT HOPE PLACE,10457,0,2375,1825,1905,211000,2012-08-17,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
4,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


In [0]:
REsample2_311 = REsample2.join(data311, on = 'zip_code')
REsample2_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
217600,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-12-20,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217601,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-03-27,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217603,2,BAYCHESTER,4715,22,NaN,A5,1156 EAST 221 STREET,10469,0,1842,1744,1950,180000,2012-12-19,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217612,2,BAYCHESTER,4730,10,NaN,A1,1320 EAST 222 STREET,10469,0,1938,1974,1901,232000,2012-08-23,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217614,2,BAYCHESTER,4730,45,NaN,A5,3418 GUNTHER AVENUE,10469,0,1710,1890,1950,335000,2012-12-19,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0


In [0]:
REsample3_311 = REsample3.join(data311, on = 'zip_code')
REsample3_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
204018,4,LAURELTON,12941,94,NaN,A2,131-43 FRANCIS LEWIS BLVD,11413,0,3560,1473,1940,0,2012-01-01,0.001934,0.0,0.002462,0.000000,0.000000,0.001055,0.000352,0.000352,0.0,0.0,0.000352,0.039388,0.000703,0.000176,0.000000,0.000000,0.0,0.0,0.038685,0.000000,0.000176,0.0,0.000176,0.0,0.018639,0.008089,...,0.000879,0.0,0.058555,0.001934,0.000000,0.000528,0.006858,0.000176,0.000528,0.0,0.0,0.0,0.002286,0.0,0.066292,0.101284,0.003165,0.002813,0.004924,0.0,0.000176,0.0,0.001055,0.002462,0.022859,0.0,0.0,0.000879,0.0,0.000528,0.000000,0.000000,0.004396,0.000000,0.000176,0.007209,0.001231,0.067698,0.000000,0.0
217598,2,BAYCHESTER,4707,70,NaN,A5,1134 EAST 213 STREET,10469,0,2320,1813,1950,335700,2012-05-17,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217602,2,BAYCHESTER,4711,96,NaN,A5,1323 HICKS STREET,10469,0,1995,1701,1950,357500,2012-05-04,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217604,2,BAYCHESTER,4715,56,NaN,A9,3840 LACONIA AVENUE,10469,0,2320,1690,1950,290000,2012-12-18,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217606,2,BAYCHESTER,4717,26,NaN,A2,1460 OAKLEY STREET,10469,0,4750,2399,1930,375000,2012-11-29,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


#### Q3. Run the linear model with 311
We need to add 311 features in addition to the gross_sq_feet,land_sq_feet,year_built used before. So regression will include both - house characteristics as well as 311 features of the location
Train the model over the training sample REsample1_311 and test over the test sample REsample3_311,
report training and test R2

In [0]:
REsample1_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,...,F122,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161
0,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
1,2,MOUNT HOPE/MOUNT EDEN,2790,53,NaN,A1,1693 CLAY AVENUE,10457,0,3610,1998,1901,168324,2012-06-28,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
2,2,MOUNT HOPE/MOUNT EDEN,2791,36,NaN,A1,1690 MONROE AVENUE,10457,0,2375,1617,1910,250000,2012-12-28,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
3,2,MOUNT HOPE/MOUNT EDEN,2802,33,NaN,A1,258 MOUNT HOPE PLACE,10457,0,2375,1825,1905,211000,2012-08-17,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
4,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


In [0]:
'sale_price ~ gross_sq_feet+land_sq_feet+year_built'+'+'+'+'.join(REsample1_311.columns[(len(REsample1_311.columns)-len(data311.columns)):])

'sale_price ~ gross_sq_feet+land_sq_feet+year_built+F0+F1+F2+F3+F4+F5+F6+F7+F8+F9+F10+F11+F12+F13+F14+F15+F16+F17+F18+F19+F20+F21+F22+F23+F24+F25+F26+F27+F28+F29+F30+F31+F32+F33+F34+F35+F36+F37+F38+F39+F40+F41+F42+F43+F44+F45+F46+F47+F48+F49+F50+F51+F52+F53+F54+F55+F56+F57+F58+F59+F60+F61+F62+F63+F64+F65+F66+F67+F68+F69+F70+F71+F72+F73+F74+F75+F76+F77+F78+F79+F80+F81+F82+F83+F84+F85+F86+F87+F88+F89+F90+F91+F92+F93+F94+F95+F96+F97+F98+F99+F100+F101+F102+F103+F104+F105+F106+F107+F108+F109+F110+F111+F112+F113+F114+F115+F116+F117+F118+F119+F120+F121+F122+F123+F124+F125+F126+F127+F128+F129+F130+F131+F132+F133+F134+F135+F136+F137+F138+F139+F140+F141+F142+F143+F144+F145+F146+F147+F148+F149+F150+F151+F152+F153+F154+F155+F156+F157+F158+F159+F160+F161'

In [0]:
resample1311preds = [[] for i in range(REsample1_311.shape[0])]
#formula = 'sale_price ~ gross_sq_feet+land_sq_feet+year_built'+'+'+'+'.join(REsample1_311.columns[14:])
lms  = smf.ols(formula = 'sale_price ~ gross_sq_feet+land_sq_feet+year_built'+'+'+'+'.join(REsample1_311.columns[(len(REsample1_311.columns)-len(data311.columns)):]), data = REsample1_311).fit()
REsample1_311['Prediction'] = lms.predict()
print(lms.summary())

                            OLS Regression Results                            
Dep. Variable:             sale_price   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     30.23
Date:                Sat, 16 Nov 2019   Prob (F-statistic):               0.00
Time:                        05:44:03   Log-Likelihood:                -85271.
No. Observations:                5710   AIC:                         1.708e+05
Df Residuals:                    5560   BIC:                         1.718e+05
Df Model:                         149                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.873e+08    6.1e+07     -3.071

In [0]:
#R2 over training
print("R2_Score for Training Data: {}".format(r2_score(REsample1_311.sale_price, lms.predict())))


R2_Score for Training Data: 0.447555858667


In [0]:
REsample3_311['test_PREDICTIONS'] =lms.predict(REsample3_311)


In [0]:
print("R2 over Testing is {}".format(((REsample3_311.test_PREDICTIONS-REsample3_311.sale_price)**2).sum()/((REsample3_311.sale_price-REsample3_311.sale_price.mean())**2).sum()))

R2 over Testing is 0.719305825508


#### Q4. Apply regularization
Train Ridge and Lasso over REsample1_311 with an arbitrary default parameter $\alpha=1$.
Then evaluate their out-of-sample performance over REsample3_311
Make sure to scale the features in all three samples first, learning the scaler over the training sample

In [0]:


num_cols = REsample1_311.columns[REsample1_311.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
REsample1_311[num_cols] = preprocessing.scale(REsample1_311[num_cols])

num_cols1 = REsample2_311.columns[REsample2_311.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
REsample2_311[num_cols1] = preprocessing.scale(REsample2_311[num_cols1])

num_cols2 = REsample3_311.columns[REsample3_311.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
REsample3_311[num_cols2] = preprocessing.scale(REsample3_311[num_cols2])



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
Y = REsample1_311.sale_price
REsample1_311.drop(labels = ['neighborhood','easement', 'building_class_present', 'address','sale_price', 'sale_date'], axis = 1, inplace = True)

y = REsample2_311.sale_price
REsample2_311.drop(labels = ['neighborhood','easement', 'building_class_present', 'address','sale_price', 'sale_date'], axis = 1, inplace = True)

y2 = REsample3_311.sale_price
REsample3_311.drop(labels = ['neighborhood','easement', 'building_class_present', 'address','sale_price', 'sale_date'], axis = 1, inplace = True)

In [0]:
#learn standardization parameters from training set
mu=REsample1_311.mean()
sigma=REsample1_311.std()

In [0]:
REsample1_311=(REsample1_311-mu)/sigma
REsample2_311=(REsample2_311-mu)/sigma
REsample3_311=(REsample3_311-mu)/sigma

In [0]:
REsample1_311.head()

,borough,block,lot,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,...,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161,Prediction
0,-2.111829,-0.893403,-0.192346,-1.160657,-0.164567,-0.511639,-0.363210,-1.670001,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.236953
1,-2.111829,-0.965613,-0.123657,-1.160657,-0.164567,0.027231,0.250513,-1.593465,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.081572
2,-2.111829,-0.965331,-0.229813,-1.160657,-0.164567,-0.377085,-0.156854,-1.249051,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.186984
3,-2.111829,-0.962229,-0.248546,-1.160657,-0.164567,-0.377085,0.065541,-1.440392,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.136661
4,-2.111829,-0.424600,-0.154880,-1.135636,-0.164567,-0.594794,0.052710,0.090336,1.569431,-0.33729,-0.401531,3.711176,0.104541,-0.408331,0.379982,0.276196,-0.599716,-0.300726,1.034588,0.644852,0.136099,-0.201545,0.175460,-0.677113,-0.454592,-0.043769,-0.400862,0.152536,0.485574,-0.238462,-0.747084,-0.585844,-0.790277,0.037231,-0.387273,0.109871,0.092542,-0.061416,-0.822579,-0.544910,...,-0.546864,

In [0]:
REsample1_311.head()

,borough,block,lot,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,...,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132,F133,F134,F135,F136,F137,F138,F139,F140,F141,F142,F143,F144,F145,F146,F147,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157,F158,F159,F160,F161,Prediction
0,-2.111829,-0.893403,-0.192346,-1.160657,-0.164567,-0.511639,-0.363210,-1.670001,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.236953
1,-2.111829,-0.965613,-0.123657,-1.160657,-0.164567,0.027231,0.250513,-1.593465,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.081572
2,-2.111829,-0.965331,-0.229813,-1.160657,-0.164567,-0.377085,-0.156854,-1.249051,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.186984
3,-2.111829,-0.962229,-0.248546,-1.160657,-0.164567,-0.377085,0.065541,-1.440392,3.536481,-0.33729,-0.349858,2.023700,-0.484463,-0.288680,-1.027910,-0.246486,-0.599716,-0.300726,-0.444672,-1.188345,1.047451,-0.201545,-0.318034,-0.768745,-0.454592,-0.043769,-1.227443,-0.356445,0.235616,-0.238462,-0.747084,-0.585844,-0.794588,-0.488575,-0.387273,-0.497504,-0.111630,-0.392514,-1.623325,-1.161348,...,-0.546864,-1.756597,-1.349936,-0.413859,-0.968994,-1.098811,-0.430537,-0.410191,-0.040637,-0.037454,0.974121,-1.096778,-0.027339,-1.768292,-2.057050,-1.108863,-1.568795,-1.567228,-0.170834,-0.605308,0.201994,-0.321781,-1.113424,-1.972652,-0.124196,-0.231145,-1.280586,-0.325866,-0.704927,-0.674430,0.239992,-0.993115,-0.453754,-0.174600,-0.719157,-1.423825,-0.937235,-0.278540,-0.208687,-0.136661
4,-2.111829,-0.424600,-0.154880,-1.135636,-0.164567,-0.594794,0.052710,0.090336,1.569431,-0.33729,-0.401531,3.711176,0.104541,-0.408331,0.379982,0.276196,-0.599716,-0.300726,1.034588,0.644852,0.136099,-0.201545,0.175460,-0.677113,-0.454592,-0.043769,-0.400862,0.152536,0.485574,-0.238462,-0.747084,-0.585844,-0.790277,0.037231,-0.387273,0.109871,0.092542,-0.061416,-0.822579,-0.544910,...,-0.546864,

In [0]:
Ridge=linear_model.Ridge(fit_intercept=True, alpha=1)
Ridge.fit(REsample1_311, Y)

ValueError: ignored

In [0]:
#In sample R-Squared
YPred_IS=Ridge.predict(REsample1_311)
print("The IS R-squared of Ridge is: {0}".format(r2_score(Y,YPred_IS)))
Ridge.coef_[:10]

In [0]:
#REsample3_311.fillna(value = 0, axis = 0, inplace = True)

In [0]:
#Out sample R-Squared
YPred_OS=Ridge.predict(REsample3_311)
print("The 0S R-squared of Ridge is: {}".format(r2_score(y2,YPred_OS)))
Ridge.coef_[:10]

# Perform Lasso

In [0]:
YY = REsample2_311.sale_price
REsample2_311.drop(labels = ['neighborhood','easement', 'building_class_present', 'address','sale_price', 'sale_date'], axis = 1, inplace = True)

In [0]:

Lasso=linear_model.Lasso(fit_intercept=True, alpha=1)

Lasso.fit(REsample1_311.iloc[:,:],Y)
# In sample:
YPred_IS=Lasso.predict(REsample1_311.iloc[:,:])
print("The IS R-squared of Lasso is: {0}".format(r2_score(Y,YPred_IS)))
#Out of sample
YPred_OS=Lasso.predict(REsample3_311.iloc[:,:])
print("The OS R-squared of Lasso is: {0}".format(r2_score(y,YPred_OS)))
Lasso.coef_[:10]

#### Q5. Perform parameter selection for Lasso and Ridge over validation sample
try exponential range -exp(10)...exp(20) with 100 values in it

Visualize the validation R2 vs alpha

For the optimal alpha evaluate performance of the models over the test set

In [0]:
#Find the Alpha and report best test performance for Ridge/Lasso.
def Regularization_fit_alpha(model,X_train,y_train,X_valid,y_valid,alphas,Graph=False, logl=False):
    #model = 1-Ridge, 2-Lasso
    #aplhas: a list of aplha values to try
    #Graph: plot the graph of R^2 values for different alpha

    R_2_OS=[] #out of sample R2's
    
    #initialize the model to use for each given alpha
    if model==1:
        RM = lambda a: linear_model.Ridge(fit_intercept=True, alpha=a)
        model_label='Ridge'
    else:
        RM = lambda a: linear_model.Lasso(fit_intercept=True, alpha=a)
        model_label='Lasso'
    
    best_R2 = -1
    best_alpha = alphas[0]
    best_coefs = []
    
    for a in alphas: #for all alphas to try
        lm = RM(a) #assign the model
        lm.fit(X_train,y_train)  #fit the regularization model
        y_predict=lm.predict(X_valid) #compute the prediction for the validation sample 
        R_2_OS_=r2_score(y_valid,y_predict)
        R_2_OS.append(R_2_OS_)
        if R_2_OS_ > best_R2: #if current OS R2 is the best - update the best solution
            best_R2 = R_2_OS_
            best_alpha = a
            best_coefs = lm.coef_
    
    if Graph==True: #visualization of the performance depending on alpha
        plt.figure(figsize = (15,10))
        plt.title('OS-R-squared for different Alpha')
        if logl: #log-scale
            plt.xlabel('ln(Alpha)')
            l=np.log(alphas)
            bl=np.log(best_alpha)
        else:
            plt.xlabel('Alpha')
            l=alphas
            bl=best_alpha
        plt.ylim((-0.1,1))
        plt.plot(l,R_2_OS,'b',label=model_label)
        plt.legend(loc='upper right')
        plt.ylabel('R-squared')
        plt.axvline(bl,color='r',linestyle='--')

        plt.show()
    
    return [best_alpha,best_R2,best_coefs]

In [0]:
REsample2_311.fillna(value = 0, axis = 0, inplace = True)

In [0]:
alphas = np.exp(np.linspace(-10.0,20.0,100)) #exponential range of possible alphas
[alpha_optimalR, best_R2R, best_coefsR]=Regularization_fit_alpha(1,REsample1_311.iloc[:,:],Y,REsample3_311.iloc[:,:], y,alphas,Graph=True,logl=True)
print('Optimal alpha for Ridge={0}, best OS R_2={1}'.format(alpha_optimalR,best_R2R))

In [0]:
[alpha_optimalL, best_R2L, best_coefsL]=Regularization_fit_alpha(2,REsample1_311.iloc[:,:],Y,REsample3_311.iloc[:,:], y,alphas,Graph=True,logl=True)
print('Optimal alpha for Lasso={0}, best OS R_2={1}'.format(alpha_optimalL,best_R2L))

In [0]:
#now consider the test sample performanc of the best Lasso model and its coefficients

In [0]:
Lasso=linear_model.Lasso(fit_intercept=True, alpha=alpha_optimalL)

Lasso.fit(REsample1_311.iloc[:,:],Y)
# In sample:
YPred_IS=Lasso.predict(REsample1_311.iloc[:,:])
print("The IS R-squared of Lasso is: {0}".format(r2_score(Y,YPred_IS)))
#Out of sample
YPred_OS=Lasso.predict(REsample2_311.iloc[:,:])
print("The OS R-squared of Lasso is: {0}".format(r2_score(YY,YPred_OS)))
Lasso.coef_[:10]

In [0]:
Ridge=linear_model.Ridge(fit_intercept=True, alpha=alpha_optimalR)
Ridge.fit(REsample1_311.loc[:,:], Y)

#In sample R-Squared
YPred_IS=Ridge.predict(REsample1_311)
print("The IS R-squared of Ridge is: {0}".format(r2_score(Y,YPred_IS)))
#Out of sample
YPred_OS=Ridge.predict(REsample2_311.iloc[:,:])
print("The OS R-squared of Lasso is: {0}".format(r2_score(YY,YPred_OS)))
Ridge.coef_[:10]

#### Task 2. Model Mean income of the zip code population using 311 profile of the neighborhood

In [0]:
#load the data
dataIncome=pd.read_csv("/content/drive/My Drive/Untitled folder/MeanZIP.csv",index_col=0)
dataIncome=dataIncome.apply(pd.to_numeric, errors='coerce')
dataIncome=dataIncome.loc[dataIncome.Mean>0]
dataIncome.head()

In [0]:
dataIncome.isna().values.any()

In [0]:
dataIncome = dataIncome[['Mean','Median','Pop']]
dataIncome.head()

In [0]:
dataIncome.index.rename('zip_code', inplace = True)


In [0]:
#split into training (1), validation (2) and test (3) samples
np.random.seed(2018)
splitind=np.random.uniform(size=len(dataIncome))
dataIncome1=dataIncome.loc[splitind<=0.4]
dataIncome2=dataIncome.loc[(splitind>0.4)&(splitind<0.7)]
dataIncome3=dataIncome.loc[(splitind>=0.7)]

In [0]:
dataIncome1.isna().values.any()

In [0]:
train = dataIncome1.merge(data311, on = 'zip_code') #Training
validation = dataIncome2.merge(data311, on = 'zip_code') #Validation
test = dataIncome3.merge(data311, on = 'zip_code') #Testing

In [0]:
train.isna().values.any()

#### Q1. Perform regularied regression of Mean income of the zip code vs its 311 profile 
Train Ridge and Lasso over dataIncome1 with an arbitrary default parameter $\alpha=100$.
Then evaluate their out-of-sample performance over dataIncome3
Make sure to scale the features in all three samples first, learning the scaler over the training sample

In [0]:
#learn standardization parameters from training set
mu=train.iloc[:,1:].mean()
sigma=train.iloc[:,1].std()

In [0]:
train.iloc[:,1:]=(train.iloc[:,1:]-mu)/sigma
validation.iloc[:,1:]=(train.iloc[:,1:]-mu)/sigma
test.iloc[:,1:]=(train.iloc[:,1:]-mu)/sigma

In [0]:
'''
y = train['Mean']
train.drop(columns='Mean', inplace = True)

y1 = validation['Mean']
validation.drop(columns='Mean', inplace = True)

y2 = test['Mean']
test.drop(columns='Mean', inplace = True)
'''


In [0]:
'''
train.fillna(value = 0, axis = 0, inplace = True)



train.dropna(how = 'any', inplace = True)
validation.dropna(how = 'any', inplace = True)
test.dropna(how = 'any', inplace = True)
'''
validation.fillna(value = 0, axis = 0, inplace = True)
test.fillna(value = 0, axis = 0, inplace = True)

In [0]:
'''
validation.replace([np.inf, -np.inf], np.nan)
validation.dropna(how = 'any', inplace = True)
''' 

In [0]:
Lasso=linear_model.Lasso(fit_intercept=True, alpha=100)

Lasso.fit(train.iloc[:,1:],train['Mean'])
# In sample:
isample=Lasso.predict(train.iloc[:,1:])
print("The IS R-squared of Lasso is: {0}".format(r2_score(train['Mean'],isample)))


#Out of sample
osample=Lasso.predict(validation.iloc[:,1:])
print("The OS R-squared of Lasso is: {0}".format(r2_score(validation['Mean'],osample)))
Lasso.coef_[:10]

In [0]:
Ridge=linear_model.Ridge(fit_intercept=True, alpha=100)
Ridge.fit(train.iloc[:,1:],train['Mean'])

#In sample R-Squared
isample=Ridge.predict(train.iloc[:,1:])
print("The IS R-squared of Ridge is: {0}".format(r2_score(train['Mean'],isample)))
#Out of sample
osample=Ridge.predict(validation.iloc[:,1:])
print("The OS R-squared of Lasso is: {0}".format(r2_score(validation['Mean'],osample)))
Ridge.coef_[:10]

#### Q2. Perform parameter selection for Lasso and Ridge over validation sample
try exponential range exp(-10)...exp(20) with 100 values in it

Visualize the validation R2 vs alpha

For the optimal alpha evaluate performance of the models over the test set. 

Why do you think performance does not look that good, especially for one of the models?

In [0]:
alphas = np.exp(np.linspace(-10.0,20.0,100)) #exponential range of possible alphas
[alpha_optimal, best_R2, best_coefs]=Regularization_fit_alpha(1,train.iloc[:,1:],train['Mean'],test.iloc[:,1:], test['Mean'],alphas,Graph=True,logl=True)
print('Optimal alpha for Ridge={0}, best OS R_2={1}'.format(alpha_optimal,best_R2))

In [0]:
alphas = np.exp(np.linspace(-10.0,20.0,100)) #exponential range of possible alphas
[alpha_optimal, best_R2, best_coefs]=Regularization_fit_alpha(2,train.iloc[:,1:],train['Mean'],test.iloc[:,1:], test['Mean'],alphas,Graph=True,logl=True)
print('Optimal alpha for Ridge={0}, best OS R_2={1}'.format(alpha_optimal,best_R2))